In [1]:
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

from flask import Flask, jsonify
import datetime as dt

In [2]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite", connect_args={'check_same_thread': False}, echo=True)
Base = automap_base()
Base.prepare(engine, reflect=True)

Measurement = Base.classes.measurement
Station = Base.classes.station
session = Session(engine)

2019-05-06 16:20:45,787 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-05-06 16:20:45,791 INFO sqlalchemy.engine.base.Engine ()
2019-05-06 16:20:45,791 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-05-06 16:20:45,791 INFO sqlalchemy.engine.base.Engine ()
2019-05-06 16:20:45,795 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-05-06 16:20:45,799 INFO sqlalchemy.engine.base.Engine ()
2019-05-06 16:20:45,803 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("measurement")
2019-05-06 16:20:45,807 INFO sqlalchemy.engine.base.Engine ()
2019-05-06 16:20:45,811 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'measurement' AND type = 'table'
2019-05-06 16:20:45,811 INFO sqlalchemy.engine.base.Engine ()
2019-05-06 16:20:45,815 INFO sqlalchemy.

In [3]:
app = Flask(__name__)

In [4]:
@app.route("/")
def welcome():
    """List all available api routes."""
    return
    """
    <html>
    <h1>List of all available Honolulu, HI API routes</h1>
    <ul>
    <br>
    <li>
    Return a list of precipitations from last year:
    <br>
    <a href="/api/v1.0/precipitation">/api/v1.0/precipitation</a>
    </li>
    <br>
    <li>
    Return a JSON list of stations from the dataset: 
    <br>
   <a href="/api/v1.0/stations">/api/v1.0/stations</a>
   </li>
    <br>
    <li>
    Return a JSON list of Temperature Observations (tobs) for the previous year:
    <br>
    <a href="/api/v1.0/tobs">/api/v1.0/tobs</a>
    </li>
    <br>
    <li>
    Return a JSON list of tmin, tmax, tavg for the dates greater than or equal to the date provided:
    <br>Replace &ltstart&gt with a date in Year-Month-Day format.
    <br>
    <a href="/api/v1.0/2017-01-01">/api/v1.0/2017-01-01</a>
    </li>
    <br>
    <li>
    Return a JSON list of tmin, tmax, tavg for the dates in range of start date and end date inclusive:
    <br>
    Replace &ltstart&gt and &ltend&gt with a date in Year-Month-Day format. 
    <br>
    <br>
    <a href="/api/v1.0/2017-01-01/2017-01-07">/api/v1.0/2017-01-01/2017-01-07</a>
    </li>
    <br>
    </ul>
    </html> 
    
    """

In [5]:
@app.route("/api/v1.0/precipitation")
def precipitation():

    """Return a list of precipitations from last year"""
    max_date = (session
                .query(Measurement.date)
                .order_by(Measurement.date.desc())
                .first())

    max_date = max_date[0]

    year_ago = dt.datetime.strptime(max_date, "%Y-%m-%d") - dt.timedelta(days=366)
    
    results_precipitation = (session
                             .query(Measurement.date, Measurement.prcp)
                             .filter(Measurement.date >= year_ago)
                             .all())

    precipitation_dict = dict(results_precipitation)

    return jsonify(precipitation_dict)

In [6]:
@app.route("/api/v1.0/stations")
def stations(): 

    """Return a JSON list of stations from the dataset."""
    results_stations =  (session
                         .query(Measurement.station)
                         .group_by(Measurement.station)
                         .all())

    stations_list = list(np.ravel(results_stations))

    return jsonify(stations_list)

In [7]:
@app.route("/api/v1.0/tobs")
def tobs(): 
  
    """Return a JSON list of Temperature Observations (tobs) for the previous year."""

    max_date = (session
                .query(Measurement.date)
                .order_by(Measurement.date.desc())
                .first())

    max_date = max_date[0]

    year_ago = dt.datetime.strptime(max_date, "%Y-%m-%d") - dt.timedelta(days=366)
    
    results_tobs = (session
                    .query(Measurement.date, Measurement.tobs)
                    .filter(Measurement.date >= year_ago)
                    .all())

    tobs_list = list(results_tobs)

    return jsonify(tobs_list)

In [8]:
@app.route("/api/v1.0/<start>")
def start(start=None):

    """Return a JSON list of tmin, tmax, tavg for the dates greater than or equal to the date provided"""

    from_start = (session
                  .query(Measurement.date, func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs))
                  .filter(Measurement.date >= start)
                  .group_by(Measurement.date)
                  .all())
    from_start_list=list(from_start)
    return jsonify(from_start_list)

In [9]:
@app.route("/api/v1.0/<start>/<end>")
def start_end(start=None, end=None):
    
    """Return a JSON list of tmin, tmax, tavg for the dates in range of start date and end date inclusive"""
    
    between_dates = (session
                     .query(Measurement.date, func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs))
                     .filter(Measurement.date >= start)
                     .filter(Measurement.date <= end)
                     .group_by(Measurement.date)
                     .all())
    between_dates_list=list(between_dates)
    return jsonify(between_dates_list)

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1

C:\Users\setho\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
